In [2]:
import pandas as pd

In [3]:
pd.read_csv("data.csv") 

/Users/ayatturar/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,3,6,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,...,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER
0,1,1999,1,1,20,2,02,34,UU,1,...,01,06,1990,01,M,41,11,1,UU,1
1,2,1999,1,1,20,2,02,34,UU,1,...,02,01,1987,01,M,19,11,1,UU,1
2,3,1999,1,1,20,2,02,34,UU,1,...,02,01,1987,02,F,20,13,2,02,2
3,4,1999,1,1,08,2,01,01,UU,5,...,01,01,1986,01,M,46,11,1,UU,1
4,5,1999,1,1,08,2,01,01,UU,5,...,99,NN,NNNN,01,M,05,99,2,UU,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,499996,2000,3,5,21,2,2,33,02,2,...,1,01,1989,02,U,UU,13,1,UU,2
499996,499997,2000,3,5,21,2,2,33,02,2,...,1,01,1989,03,M,22,23,2,02,2
499997,499998,2000,3,5,21,2,2,33,02,2,...,2,01,1992,01,M,35,11,1,02,1
499998,499999,2000,3,5,21,2,2,33,02,2,...,2,01,1992,02,M,15,13,1,02,2


In [ ]:
# Read in CSV file

In [ ]:
# Data cleaning: P_ISEV

In [ ]:
# Data cleaning: P_SEX

In [ ]:
# Data cleaning: C_MNTH

In [ ]:
# Data cleaning: C_WDAY

In [ ]:
# Data cleaning: C_HOUR

In [ ]:
# Data cleaning: C_VEHS

In [ ]:
# Data cleaning: C_CONF

In [ ]:
# Data cleaning: C_RCFG

In [ ]:
# Data cleaning: C_WTHR

In [ ]:
# Data cleaning: C_RSUR

In [ ]:
# Data cleaning: C_RALN 

In [2]:
# Data cleaning: C_TRAF 

In [3]:
# Data cleaning: V_ID 

In [ ]:
# Data cleaning: V_TYPE

In [ ]:
# Data cleaning: V_YEAR

In [ ]:
# Data cleaning: P_ID

In [ ]:
# Data cleaning: P_PSN

In [ ]:
# Data cleaning: P_SAFE

In [ ]:
# Data cleaning: P_USER

In [ ]:
# Data cleaning: P_AGE

In [ ]:
# Write cleaned data to CSV